In [51]:
import pandas as pd
from dotenv import load_dotenv
import os
import sqlalchemy
from snowflake.sqlalchemy import URL

load_dotenv()


#set credentials what snowflake needs
engine = sqlalchemy.create_engine(URL(
                user=os.getenv("SNOWFLAKE_USER"),
                password=os.getenv("SNOWFLAKE_PWD"),
                account=os.getenv("SNOWFLAKE_ACCOUNT"),
                role=os.getenv("SNOWFLAKE_ROLE"),
                warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
                database=os.getenv("SNOWFLAKE_DATABASE"),
                schema=os.getenv("SNOWFLAKE_SCHEMA")
            ))

#basic query
query = "SELECT * FROM ECDC_GLOBAL WHERE CONTINENTEXP = 'Europe';"

tables = pd.read_sql(query, engine)
tables.head()


,country_region,continentexp,iso3166_1,cases,deaths,cases_since_prev_day,deaths_since_prev_day,population,date,last_update_date,last_reported_flag
0,Albania,Europe,AL,788.0,14.0,0.0,0.0,2862427.0,2020-12-14,2023-12-14 00:03:31.349504,True
1,Albania,Europe,AL,879.0,12.0,91.0,-2.0,2862427.0,2020-12-13,2023-12-14 00:03:31.349504,False
2,Albania,Europe,AL,802.0,12.0,-77.0,0.0,2862427.0,2020-12-12,2023-12-14 00:03:31.349504,False
3,Albania,Europe,AL,873.0,14.0,71.0,2.0,2862427.0,2020-12-11,2023-12-14 00:03:31.349504,False
4,Albania,Europe,AL,752.0,15.0,-121.0,1.0,2862427.0,2020-12-10,2023-12-14 00:03:31.349504,False


In [52]:
query_countries = """
SELECT
  COUNTRY_REGION,
  ISO3166_1,
  DATE,
  SUM(CASES) OVER (PARTITION BY COUNTRY_REGION ORDER BY DATE) AS cumulative_cases,
  SUM(DEATHS) OVER (PARTITION BY COUNTRY_REGION ORDER BY DATE) AS cumulative_deaths,
  POPULATION,
  (SUM(CASES) OVER (PARTITION BY COUNTRY_REGION ORDER BY DATE) / NULLIF(POPULATION, 0)) * 100000 AS cases_per_100k,
  (SUM(DEATHS) OVER (PARTITION BY COUNTRY_REGION ORDER BY DATE) / NULLIF(POPULATION, 0)) * 100000 AS deaths_per_100k
FROM ECDC_GLOBAL
WHERE CONTINENTEXP = 'Europe'
QUALIFY ROW_NUMBER() OVER (PARTITION BY COUNTRY_REGION, DATE ORDER BY LAST_UPDATE_DATE DESC) = 1
ORDER BY COUNTRY_REGION, DATE;
"""

countries_with_cases = pd.read_sql(query_countries, engine)

display(countries_with_cases)

,country_region,iso3166_1,date,cumulative_cases,cumulative_deaths,population,cases_per_100k,deaths_per_100k
0,Albania,AL,2020-03-09,2.0,0.0,2862427.0,0.069871,0.000000
1,Albania,AL,2020-03-10,6.0,0.0,2862427.0,0.209612,0.000000
2,Albania,AL,2020-03-11,10.0,0.0,2862427.0,0.349354,0.000000
3,Albania,AL,2020-03-12,11.0,1.0,2862427.0,0.384289,0.034935
4,Albania,AL,2020-03-13,23.0,1.0,2862427.0,0.803514,0.034935
...,...,...,...,...,...,...,...,...
17367,United_Kingdom,UK,2020-12-10,1766819.0,62566.0,66647112.0,2651.006093,93.876536
17368,United_Kingdom,UK,2020-12-11,1787783.0,63082.0,66647112.0,2682.461320,94.650763
17369,United_Kingdom,UK,2020-12-12,1809455.0,63506.0,66647112.0,2714.978858,95.286950
17370,United_Kingdom,UK,2020-12-13,1830956.0,64026.0,66647112.0,2747.239820,96.067178


In [53]:
# Query for weekly new cases and deaths per country
query_daily = """
SELECT
  country_region,
  iso3166_1,
  date,
  cases AS daily_cases,
  deaths AS daily_deaths,
  population,
  (cases / NULLIF(population, 0)) * 100000 AS cases_per_100k,
  (deaths / NULLIF(population, 0)) * 100000 AS deaths_per_100k
FROM ECDC_GLOBAL
WHERE CONTINENTEXP = 'Europe'
ORDER BY country_region, date;
"""

weekly_data = pd.read_sql(query_daily, engine)
display(weekly_data)

query_countries = """
SELECT
  country_region,
  iso3166_1,
  date,
  SUM(cases) OVER (PARTITION BY country_region ORDER BY date) AS cumulative_cases,
  SUM(deaths) OVER (PARTITION BY country_region ORDER BY date) AS cumulative_deaths,
  population,
  (SUM(cases) OVER (PARTITION BY country_region ORDER BY date) / NULLIF(population, 0)) * 100000 AS cases_per_100k,
  (SUM(deaths) OVER (PARTITION BY country_region ORDER BY date) / NULLIF(population, 0)) * 100000 AS deaths_per_100k
FROM ECDC_GLOBAL
WHERE CONTINENTEXP = 'Europe'
QUALIFY ROW_NUMBER() OVER (PARTITION BY country_region, date ORDER BY LAST_UPDATE_DATE DESC) = 1
ORDER BY country_region, date;
"""

countries_with_cases = pd.read_sql(query_countries, engine)

display(countries_with_cases)

,country_region,iso3166_1,date,daily_cases,daily_deaths,population,cases_per_100k,deaths_per_100k
0,Albania,AL,2020-03-09,2.0,0.0,2862427.0,0.069871,0.000000
1,Albania,AL,2020-03-10,4.0,0.0,2862427.0,0.139742,0.000000
2,Albania,AL,2020-03-11,4.0,0.0,2862427.0,0.139742,0.000000
3,Albania,AL,2020-03-12,1.0,1.0,2862427.0,0.034935,0.034935
4,Albania,AL,2020-03-13,12.0,0.0,2862427.0,0.419225,0.000000
...,...,...,...,...,...,...,...,...
17367,United_Kingdom,UK,2020-12-10,16578.0,533.0,66647112.0,24.874296,0.799735
17368,United_Kingdom,UK,2020-12-11,20964.0,516.0,66647112.0,31.455226,0.774227
17369,United_Kingdom,UK,2020-12-12,21672.0,424.0,66647112.0,32.517538,0.636187
17370,United_Kingdom,UK,2020-12-13,21501.0,520.0,66647112.0,32.260963,0.780229


,country_region,iso3166_1,date,cumulative_cases,cumulative_deaths,population,cases_per_100k,deaths_per_100k
0,Albania,AL,2020-03-09,2.0,0.0,2862427.0,0.069871,0.000000
1,Albania,AL,2020-03-10,6.0,0.0,2862427.0,0.209612,0.000000
2,Albania,AL,2020-03-11,10.0,0.0,2862427.0,0.349354,0.000000
3,Albania,AL,2020-03-12,11.0,1.0,2862427.0,0.384289,0.034935
4,Albania,AL,2020-03-13,23.0,1.0,2862427.0,0.803514,0.034935
...,...,...,...,...,...,...,...,...
17367,United_Kingdom,UK,2020-12-10,1766819.0,62566.0,66647112.0,2651.006093,93.876536
17368,United_Kingdom,UK,2020-12-11,1787783.0,63082.0,66647112.0,2682.461320,94.650763
17369,United_Kingdom,UK,2020-12-12,1809455.0,63506.0,66647112.0,2714.978858,95.286950
17370,United_Kingdom,UK,2020-12-13,1830956.0,64026.0,66647112.0,2747.239820,96.067178


In [54]:
#regions in the dataset, which are very small countries / regions, under 50.000 population

small_countries = weekly_data[weekly_data['population'] < 150000]

print(small_countries[['country_region', 'population']].drop_duplicates().to_string(index=False))



               country_region  population
                      Andorra     76177.0
                Faroe Islands     48677.0
                    Gibraltar     33706.0
                     Guernsey     64468.0
Holy See (Vatican City State)       815.0
                  Isle of Man     84589.0
                       Jersey    107796.0
                Liechtenstein     38378.0
                       Monaco     33085.0
                   San Marino     34453.0


In [55]:
query_daily = """
        SELECT COUNTRY_REGION, ISO3166_1, DATE, CASES AS DAILY_CASES, DEATHS AS DAILY_DEATHS, POPULATION
        FROM ECDC_GLOBAL
        WHERE CONTINENTEXP = 'Europe' AND POPULATION IS NOT NULL AND POPULATION > 0 AND DATE IS NOT NULL
        ORDER BY COUNTRY_REGION, DATE;
        """
df_daily = pd.read_sql(query_daily, engine)

# Standardize to lowercase
df_daily.columns = [col.lower() for col in df_daily.columns]
df_daily.rename(columns={'country_region': 'country_name'}, inplace=True)

POPULATION_THRESHOLD = 150000
country_populations = df_daily.groupby('country_name')['population'].max()
countries_to_keep = country_populations[country_populations >= POPULATION_THRESHOLD].index
df_daily = df_daily[df_daily['country_name'].isin(countries_to_keep)]

df_daily['daily_cases'] = df_daily['daily_cases'].clip(lower=0)
df_daily['daily_deaths'] = df_daily['daily_deaths'].clip(lower=0)
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily['daily_cases_per_100k'] = (df_daily['daily_cases'] / df_daily['population']) * 100000
df_daily['daily_deaths_per_100k'] = (df_daily['daily_deaths'] / df_daily['population']) * 100000
cols_to_fill = ['daily_cases_per_100k', 'daily_deaths_per_100k']
for col in cols_to_fill:
    df_daily[col] = df_daily[col].replace([float('inf'), -float('inf')], 0).fillna(0)

print(df_daily.head)

<bound method NDFrame.head of          country_name iso3166_1       date  daily_cases  daily_deaths  \
0             Albania        AL 2020-03-09          2.0           0.0   
1             Albania        AL 2020-03-10          4.0           0.0   
2             Albania        AL 2020-03-11          4.0           0.0   
3             Albania        AL 2020-03-12          1.0           1.0   
4             Albania        AL 2020-03-13         12.0           0.0   
...               ...       ...        ...          ...           ...   
17367  United_Kingdom        UK 2020-12-10      16578.0         533.0   
17368  United_Kingdom        UK 2020-12-11      20964.0         516.0   
17369  United_Kingdom        UK 2020-12-12      21672.0         424.0   
17370  United_Kingdom        UK 2020-12-13      21501.0         520.0   
17371  United_Kingdom        UK 2020-12-14      18447.0         144.0   

       population  daily_cases_per_100k  daily_deaths_per_100k  
0       2862427.0           

In [56]:
range_data_df = df_daily[
            (df_daily['date'] >= pd.to_datetime('2020-01-01')) &
            (df_daily['date'] <= pd.to_datetime('2020-02-02'))].copy()

map_agg_data_df = range_data_df.groupby(['country_name', 'iso3166_1']).agg(
    range_cases=('daily_cases', 'sum'), range_deaths=('daily_deaths', 'sum'),
    population=('population', 'first')
).reset_index()


In [57]:
import geopandas as gpd
from shapely.geometry import box
import os

def load_europe_map():
    local_map_path = "countries.geojson" 
    if not os.path.exists(local_map_path):
        return gpd.GeoDataFrame()
    
    world = gpd.read_file(local_map_path)
    # Convert GeoDataFrame columns to lowercase for consistency
    world.columns = [col.lower() if isinstance(col, str) else col for col in world.columns]

    europe_gdf = world[(world.continent == 'Europe') |
                    ((world.admin == 'Russia') & (world.subregion == 'Eastern Europe')) |
                    (world.admin == 'Turkey')].copy()
    
    for idx, row in europe_gdf[europe_gdf.admin == 'Russia'].iterrows():
        european_part_russia = box(-180, -90, 50, 90) 
        clipped_geometry = row.geometry.intersection(european_part_russia)
        europe_gdf.loc[idx, 'geometry'] = clipped_geometry
    
    return europe_gdf

europe_gdf = load_europe_map()

In [58]:
merged_gdf_for_plotly = europe_gdf.merge(
                    map_agg_data_df, left_on='iso_a2_eh', right_on='iso3166_1', how='inner'
                )

In [59]:
#make a sql query, that will give back the most amount of cases per capita in the whole timeframe of the dataset
query_max_cases = """
SELECT COUNTRY_REGION, ISO3166_1, SUM(CASES) AS TOTAL_CASES, POPULATION,
         (SUM(CASES) / NULLIF(POPULATION, 0)) * 100000 AS CASES_PER_100K
FROM ECDC_GLOBAL
WHERE CONTINENTEXP = 'Europe' AND POPULATION IS NOT NULL AND POPULATION > 0
GROUP BY COUNTRY_REGION, ISO3166_1, POPULATION
ORDER BY CASES_PER_100K DESC
LIMIT 10;
"""

max_cases_df = pd.read_sql(query_max_cases, engine)

print(max_cases_df[['country_region', 'iso3166_1', 'total_cases', 'population', 'cases_per_100k']].to_string(index=False))

merged_gdf_for_plotly = europe_gdf.merge(
                    map_agg_data_df, left_on='iso_a2_eh', right_on='iso3166_1', how='inner'
                )


country_region iso3166_1  total_cases  population  cases_per_100k
       Andorra        AD       7338.0     76177.0     9632.828807
    Luxembourg        LU      41272.0    613894.0     6722.984750
    Montenegro        ME      41426.0    622182.0     6658.180404
    San Marino        SM       1947.0     34453.0     5651.176966
       Czechia        CZ     581079.0  10649800.0     5456.243310
       Belgium        BE     608001.0  11455519.0     5307.494143
       Armenia        AM     148682.0   2957728.0     5026.899025
       Georgia        GE     191063.0   3996762.0     4780.444770
      Slovenia        SI      96314.0   2080908.0     4628.460268
   Switzerland        CH     372329.0   8544527.0     4357.514465
